<a href="https://colab.research.google.com/github/ShambhaviCodes/ASLRFeatureExtractor/blob/master/Document%20Similarity/DocSim_SD_Claims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import string
import contextlib
import re
import scipy
from scipy import spatial
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def load_doc(filename):
    with open(filename, 'r') as file:
        text = file.read()
    return text

In [4]:
from collections import Counter
def clean_doc(doc):
     tokens = doc.split()
     re_punc = re.compile('[%s]' % re.escape(string.punctuation))
     tokens = [re_punc.sub('', w) for w in tokens]
     tokens = [word for word in tokens if word.isalpha()]
     stopwords = nltk.corpus.stopwords.words('english')
     s211 = ['clause','Technical', 'Specification', 'For', 'The', 'There', 'first', 'One', 'Overview']
     s212 = ['The', 'bit', 'block', 'number', 'column', 'Technical', 'Clause', 'Specification', 'two', 'for', 'one', 'TS', 'Generation', 'Partnership', 'Project', 'Group', 'Radio', 'Access', 'Network', 'NR']
     stopwords.extend(s211)
     tokens = [w for w in tokens if not w in stopwords]
     tokens = [word for word in tokens if len(word) > 1]
     counts = Counter(tokens)
     print(counts)
     '''abbr211 = ['Physical random-access channel', 'Physical resource block',
	              'Primary synchronization signal',
	              'Phase-tracking reference signal',
	              'Physical uplink control channel',
	              'Physical uplink shared channel',
               	'Resource-element group',
	              'Remote interference management',
	              'Remote interference management reference signal']
     #tokens.extend(abbr)
     #print(tokens)'''
     return tokens

#### Stopwords
[Technical, Specification, For, The, There, first, One, Overview]
#### Abbreviations : 
BWP	Bandwidth part
CCE	Control channel element
CORESET	Control resource set
CRB	Common resource block
CSI	Channel-state information
CSI-RS	CSI reference signal 
DCI	Downlink Control Information
DM-RS	Demodulation reference signal
FR1	Frequency range 1 as defined in [8, TS 38.104]
FR2	Frequency range 2 as defined in [8, TS 38.104] 
IAB	Integrated access and backhaul
IAB-MT	IAB mobile termination 
IE	Information element
PBCH	Physical broadcast channel
PDCCH	Physical downlink control channel
PDSCH	Physical downlink shared channel
PRACH	Physical random-access channel 
PRB	Physical resource block
PSS	Primary synchronization signal
PT-RS	Phase-tracking reference signal
PUCCH	Physical uplink control channel
PUSCH	Physical uplink shared channel
REG	Resource-element group
RIM	Remote interference management
RIM-RS	Remote interference management reference signal
SRS	Sounding reference signal
SSS	Secondary synchronization signal
VRB	Virtual resource block

In [5]:
def clean_query(doc):
     tokens = doc.split()
     re_punc = re.compile('[%s]' % re.escape(string.punctuation))
     tokens = [re_punc.sub('', w) for w in tokens]
     tokens = [word for word in tokens if word.isalpha()]
     stop_words = set(stopwords.words('english'))
     tokens = [w for w in tokens if not w in stop_words]
     tokens = [word for word in tokens if len(word) > 1]
     return tokens

In [6]:
!pip install fasttext
import os
import fasttext

     |████████████████████████████████| 71kB 3.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3018142 sha256=b8e537111425e4c1840f075a66068da00c76fbbed316be8bbe1fad83a60d8991
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [7]:
#Calculation of similarity score between the base_document and query
def cosine_similarity_score(base_document, query):
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        model = fasttext.load_model('/content/drive/My Drive/Patent_RW/cc.en.300.bin')
    base_vector = np.mean([model[word] for word in base_document],axis=0)
    query_vector = np.mean([model[word] for word in query],axis=0)
    cosine = scipy.spatial.distance.cosine(base_vector, query_vector)
    return (round((1-cosine)*100,2),'%')

In [8]:
doc = load_doc('/content/drive/My Drive/Patent_RW/38211-g10.txt')
base_document = clean_doc(doc)

Counter({'resource': 328, 'given': 270, 'block': 220, 'shall': 220, 'sequence': 208, 'physical': 178, 'parameter': 168, 'higherlayer': 165, 'number': 152, 'symbol': 151, 'DMRS': 150, 'Table': 147, 'transmission': 144, 'according': 141, 'TS': 134, 'symbols': 125, 'defined': 121, 'blocks': 120, 'configured': 120, 'resources': 117, 'within': 115, 'UE': 109, 'reference': 106, 'Mapping': 105, 'mapping': 102, 'OFDM': 101, 'index': 100, 'set': 94, 'slot': 93, 'antenna': 90, 'PUSCH': 89, 'subcarrier': 89, 'used': 86, 'frequency': 85, 'signals': 79, 'signal': 78, 'generation': 74, 'type': 74, 'PUCCH': 72, 'order': 72, 'mapped': 72, 'elements': 70, 'channel': 70, 'PDSCH': 70, 'Sequence': 68, 'Physical': 67, 'format': 63, 'port': 63, 'hopping': 62, 'starting': 62, 'downlink': 61, 'otherwise': 61, 'modulation': 60, 'assume': 60, 'ports': 57, 'configuration': 56, 'one': 56, 'SSPBCH': 55, 'RIMRS': 54, 'uplink': 54, 'bandwidth': 54, 'precoding': 52, 'spacing': 52, 'time': 52, 'IE': 52, 'increasing': 

In [10]:
queries = pd.read_csv('/content/drive/My Drive/Patent_RW/DocSim.csv')
queries_df = pd.DataFrame(queries) 

tokenized_queries = [clean_doc(text) for text in queries['Independent Claims']]

Counter({'network': 22, 'device': 21, 'authentication': 20, 'information': 17, 'access': 14, 'evolved': 14, 'packet': 14, 'core': 14, 'computing': 14, 'server': 11, 'receiving': 9, 'system': 9, 'wireless': 8, 'local': 8, 'area': 8, 'node': 8, 'communication': 8, 'memory': 8, 'authorization': 7, 'stored': 7, 'accounting': 6, 'request': 6, 'comprising': 5, 'specifying': 5, 'limits': 4, 'processor': 4, 'storing': 3, 'corresponding': 3, 'provisioned': 3, 'accessible': 3, 'via': 3, 'determining': 3, 'response': 3, 'required': 3, 'communicating': 3, 'connection': 3, 'authenticating': 3, 'establishing': 3, 'providing': 3, 'subscriber': 2, 'identity': 2, 'retrieving': 2, 'retrieved': 2, 'communicatively': 2, 'coupled': 2, 'therein': 2, 'executable': 2, 'instructions': 2, 'performing': 2, 'operations': 2, 'searching': 2, 'computer': 1, 'implemented': 1, 'method': 1, 'received': 1, 'IMSI': 1, 'entity': 1, 'responsible': 1, 'charges': 1, 'associated': 1})
Counter({'UE': 31, 'data': 18, 'downlink'

In [11]:
score = []
for tokens_query in tokenized_queries:
  score.append(cosine_similarity_score(base_document, tokens_query))

In [14]:
queries_df['Independent Claims Score'] = score
queries_df['IC Tokens'] = tokenized_queries

In [15]:
#Saving output in (.CSV) format
queries_df.to_csv('DocSim.csv', header=False, index=False)